In [1]:
import tensorflow as tf 
from tensorflow.keras.models import load_model 
import pickle 
import pandas as pd 
import numpy as np

### Load the trainedmodel , scaler pickle , onehot pickle 

In [7]:
model =load_model('model.h5')

with open('onehot_encoder_geography.pkl','rb') as file:
    onehot_encoder_geography=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)


In [17]:
 ### Example input data 
input_data={'CreditScore':600,
             'Geography':'France',
             'Gender':'Male',
             'Age':40,
             'Tenure':3,
             'Balance':60000,
             'NumOfProducts': 2,
             'HasCrCard':1,
             'IsActiveMember': 1,
             'EstimatedSalary': 50000}

In [18]:
### One hot encode 'Geography'
geo_encoded=label_encoder_geography.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geography.get_feature_names_out(['Geography']))

C:\Users\mishraha\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [19]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [20]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [21]:
### Encode Categorical variables 
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


### Concatenation on one hot encoded


In [22]:
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)

In [23]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


### Scaling the input data 


In [24]:
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

### Prediction

In [25]:
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step


array([[0.04269691]], dtype=float32)

In [26]:
prediction_probability=prediction[0][0]
prediction_probability

0.04269691

In [27]:
if prediction_probability>0.5:
    print('The Customer is likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
